# IA - Le compte est bon
First have a look at the instructions of the lab:
http://www.ai-junkie.com/ga/intro/gat3.html

## Getting started
To have the following code working you will need to install the DEAP framework (https://github.com/deap/deap).
We encourage you to use **_pip_**:

`pip install deap`

If you use conda you can try:
`conda install -c conda-forge deap`

_Note: Linux package managers like apt-get, yum, etc. usually provide an outdated version._

In [86]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import operator
from enum import Enum
from collections import namedtuple
import random
import time

## Private part
_=== No need to change something here. ===_  

You can find here constants and private functions useful for the lab. 

Remember, four bits are required to represent the range of characters used:

0:&nbsp; &nbsp; &nbsp;0000  
1:&nbsp; &nbsp; &nbsp;0001  
2:&nbsp; &nbsp; &nbsp;0010  
3:&nbsp; &nbsp; &nbsp;0011  
4:&nbsp; &nbsp; &nbsp;0100  
5:&nbsp; &nbsp; &nbsp;0101  
6:&nbsp; &nbsp; &nbsp;0110  
7:&nbsp; &nbsp; &nbsp;0111  
8:&nbsp; &nbsp; &nbsp;1000  
9:&nbsp; &nbsp; &nbsp;1001  
+:&nbsp; &nbsp; &nbsp;1010  
-:&nbsp; &nbsp; &nbsp;1011  
*:&nbsp; &nbsp; &nbsp;1100  
/:&nbsp; &nbsp; &nbsp;1101  

In [87]:
# Each operand or operator is described by 4 bits
CODE_LENGTH = 4

# In this example, we fix the number of operands to 5
NB_OPERANDS = 5

# The maximum number of operators is NB_OPERANDS - 1
# ex. 5 + 3 / 2 
# three operands: 5, 3, 2
# two operators: +, /
NB_OPERATORS = NB_OPERANDS - 1


CHROMOSOME_LENGTH = NB_OPERANDS * CODE_LENGTH + NB_OPERATORS * CODE_LENGTH

# We have three types of code: operands, operators and undefined symbols
class CodeType(Enum):
    OPERAND = 1
    OPERATOR = 2
    NOTHING = 3

# namedtuple("typename, field_names[...]") returns a new tuple subclass named 'typename'. 
# The new subclass is used to create tuple-like objects that have fields accessible 
# by attribute lookup as well as being indexable and iterable
Code = namedtuple("Code", ["code_type", "apply", "str"])

OPERATORS = {
    10: (operator.add, "+"),  # Standard operators as functions, see https://docs.python.org/3/library/operator.html
    11: (operator.sub, "-"),
    12: (operator.mul, "*"),
    13: (operator.truediv, "/")
}

def _parse_code(code):
    """ Convert bit string to a Code namedtuple """
    int_value = int(code, 2)
    if int_value >= 0 and int_value < 10:
        return Code(CodeType.OPERAND, lambda: int_value, str(int_value))
    elif int_value >= 10 and int_value <= 13:
        return Code(CodeType.OPERATOR, OPERATORS[int_value][0], OPERATORS[int_value][1])
    else:
        return Code(CodeType.NOTHING, None, "_")
    
def _decode(individual):
    """ Parse each code of the full chromosome (aka individual) """
    chromosome_str = "".join([str(gene) for gene in individual])
    codes = [_parse_code(chromosome_str[i: i + CODE_LENGTH]) for i in range(0, len(chromosome_str), CODE_LENGTH)]
    return codes

## Public part
_=== No need to change something here. ===_  

You find here functions that may be interesting for you. 

In [88]:
def display_chromosome(individual):
    """ Convert chromosome to a readable format (e.g. 3 + 5 / 6) """
    return " ".join(code.str for code in _decode(individual))

def compute_chromosome(individual):
    """ Compute operations described by a the chromosome """
    codes = _decode(individual)
    first_operand = None
    operation = None
    snd_operand = None
    result = 0
    for code in codes:
        if not first_operand:
            if code.code_type == CodeType.OPERAND:
                first_operand = code.apply()
        elif not operation:
            if code.code_type == CodeType.OPERATOR:
                operation = code.apply
        elif not snd_operand:
            if code.code_type == CodeType.OPERAND:
                snd_operand = code.apply()
                try:
                    result = operation(first_operand, snd_operand)
                except ZeroDivisionError:
                    pass
                first_operand = result
                operation = None
                snd_operand = None
    return result

## Deap framework
_=== No need to change something here. ===_  

You find here a preparation of tools necessary for our algorithm. 

In [89]:
toolbox = base.Toolbox()

The **fitness** will measure the proximity between the result of the chromosome and a target value.
The **lower the proximity is - the better is our chromosome** so we are in a minimization problem. 

/!\ Be aware that `values` and `weights` must be tuples.

First we create the code of our fitness function; then we add it into our toolbox. 

In [90]:
def fitness(individual, target):
    return (abs(compute_chromosome(individual) - target),) # Tuple !

toolbox.register("fitness", fitness)

The following line creates, in the `creator`, a ready to use single objective minimizing fitness named _FitnessMin_.

`base.Fitness`: if values are provided as a tuple, the fitness is initalized using those values, otherwise it is empty (or invalid).  
`weights` is used to define a maximization/minimization by setting 1.0 or -1.0.

In [91]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))


`deap.creator.create(name, base[, attribute[, ...]])`  
The `create()` function takes at least two arguments, a name for the newly created class and a base class. Any subsequent argument becomes an attribute of the class. 

A `deep.creator.Individual` will be list of genes with an attribute 'fitness' of type `deep.creator.FitnessMin` just created.

In [92]:
creator.create("Individual", list, fitness=creator.FitnessMin) 

### TODO 1:
Look at the documentation of the DEAP framework and register into the toolbox the following tools:
- **Crossover** between two individuals will be a simple one point crossover.
- **Mutation** of a individual will flip the bit in the gene with a probability of 10%.
- **Selection** of k individuals will be done using a tournament.
Then, complete the following cell:

In [93]:
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament)

Finally we provide initiation operations. A **population** will be a list of `deep.creator.Individual`. Each gene will be set randomly to 0 or 1.

In [94]:
toolbox.register("init_gene", random.randint, 0, 1)
toolbox.register("init_individual", tools.initRepeat, creator.Individual, toolbox.init_gene, CHROMOSOME_LENGTH)
toolbox.register("init_population", tools.initRepeat, list, toolbox.init_individual)

## TODO 2: Solve "Le compte est bon"
It'y sour turn! Using Deap and previous tools design a loop to obtain **TARGET** value in a maximum time of **MAX_TIME**.
- Create the **evaluate_population()** method to calculate the fitness of the population
- Break the loop if an individual is optimal before **MAX_TIME** (i.e. his fitness = 0).
- Display the best chromosome
- Display the total time


In [95]:
def evaluate_population(population, target):
    for ind in population:
        ind.fitness.values = toolbox.fitness(ind, target)

toolbox.register("evaluate", evaluate_population)

def find_winners(population):
    winners = [i for i in population if i.fitness.values[0] == 0]
    return winners

In [96]:
TARGET = 126
MAX_TIME = 25  # seconds

MUTPB = 0.2
CXPB = 0.7
tournSize = 8
solution = None

In [97]:
#init pop
pop = toolbox.init_population(n=10)
#give base fitness
toolbox.evaluate(pop, TARGET)
#init time
start = time.time()
timePassed=0
iterations = 0

#while loop
while timePassed < MAX_TIME and len(find_winners(pop)) <= 0 :
    # --- SEL ---
    offspring = toolbox.select(pop, len(pop), tournSize)
    offspring = list(map(toolbox.clone, offspring))

    # --- MATE&MUTATE ---
    #mate
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
    #mutate
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)

    
    #remplace pop
    pop = offspring
    #pop eval
    toolbox.evaluate(pop, TARGET)

    # Search for the solution
    fitnesses = [ind.fitness.values[0] for ind in pop]
    min_fit = min(fitnesses)
    best = pop[fitnesses.index(min_fit)]
    if not solution or best.fitness.values[0] < solution.fitness.values[0]:
        solution = best

    #calculate end time
    timePassed = time.time() - start
    iterations += 1

winner = solution if solution is not None else find_winners(pop)
if winner :
    print(f"winner : {winner}")
    print(f"found in {iterations} iterations and {timePassed}s")
    print(f"Solution : {winner} = {displayable_chromosome(winner)}, computed : {compute_chromosome(winner)}, value : {winner.fitness}")
else :
    print(f"Solution not found")

winner : [1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
found in 34 iterations and 0.03490161895751953s


NameError: name 'displayable_chromosome' is not defined

## TODO 3: Advanced - Find the best Hyperparameters (optional)
- Which are the **best**:
    - population size  
    - frequence of mutation
    - frequence of crossover
    - ...
- Implement the elitism, the best N individuals will surely survive (N = 2) and without undergoing mutation